# Evaluierung der generierten Ausdrücke mit Sympy

In [3]:
from sympy import *
import numpy as np
from itertools import product, combinations_with_replacement, chain
from operator import itemgetter, add
x,y = symbols('x y')

## Definitions- und Wertebereich

Für den **Definitionsbereich** kann die Funktion `continuous_domain` genutzt werden:

In [17]:
f = sympify("Add(Mul(Symbol('c'), Pow(Symbol('x'), Integer(-1))), Mul(Symbol('c'), Pow(Symbol('x'), Integer(-1)), Pow(sin(Symbol('x')), Integer(-1))), Integer(-1))")
d = calculus.util.continuous_domain(f, x, S.Reals)
print(d)

Complement(Reals, Union(ImageSet(Lambda(_n, 2*_n*pi + pi), Integers), ImageSet(Lambda(_n, 2*_n*pi), Integers)))


Für den **Wertebereich** gibt es die Funktion `function_range`, diese wirft allerdings einen Fehler falls dieser nicht berechenbar ist:

In [16]:
f = (x+2)*(x+1)
r = calculus.util.function_range(f, x, S.Reals)
print(r)

Interval(-1/4, oo)


In [18]:
f = sin(x)/x
try:
  calculus.util.function_range(f, x, S.Reals)
except NotImplementedError:
  print('Not able to calculate range')

Not able to calculate range


### Unstetigkeitsstellen

Mit der Funktion `singularities` können Unstetigkeitsstellen gefunden werden, allerdings wurde noch keine Funktion gefunden, die herausfinden kann um welche Art Unstetigkeitsstelle es sich dabei handelt.

Weiterhin kann auch diese Funktion einen Fehler werfen, falls diese nicht berechenbar sind.

In [49]:
f = 1+4*(x**2-1)/x**4
singularities(f, x)

### Periodizität

Hierfür gibt es die Funktion `periodicity`

In [8]:
f = sympify("Add(Mul(Symbol('c'), Pow(Symbol('x'), Integer(-1))), Mul(Symbol('c'), Pow(Symbol('x'), Integer(-1)), Pow(sin(Symbol('x')), Integer(-1))), Integer(-1))")
periodicity(f, x)

## y-Achsenschnitt & Nullstellen

### Nullstellen

Nullstellen können einfach mit `roots` berechnet werden. Diese haben dann die Form {Nullstelle: Ordnung} - dadurch kann gleichzeitig festgestellt werden, ob die Nullstelle ein Wende- oder Sattelpunkt ist.

In [15]:
f = sin(x)
str(solve(f, x))

'[0, pi]'

### y-Achsenschnitt

Für den y-Achsenschnitt kann einfach für x = 0 eingesetzt werden.

In [23]:
f = sympify("Add(Mul(Integer(2), Pow(Symbol('x'), Integer(-1))), Mul(Integer(2), Pow(Symbol('x'), Integer(-1)), Pow(sin(Symbol('x')), Integer(-1))), Integer(-1))")
f.subs(x, 0)

nan

## Ableitungen

Die Ableitung kann mit Hilfe von `diff` gebildet werden.

In [24]:
f = sympify("Add(Mul(Integer(2), Pow(Symbol('x'), Integer(-1))), Mul(Integer(2), Pow(Symbol('x'), Integer(-1)), Pow(sin(Symbol('x')), Integer(-1))), Integer(-1))")
diff(f, x)

-2*cos(x)/(x*sin(x)**2) - 2/x**2 - 2/(x**2*sin(x))

### Lokale & globale Minima/Maxima & Wendepunkte

Dies kann mit Hilfe von `solveset` auf den Ableitungen der Funktion berechnet werden.

In [28]:
f = sympify("Add(Mul(Integer(2), Pow(Symbol('x'), Integer(-1))), Mul(Integer(2), Pow(Symbol('x'), Integer(-1)), Pow(sin(Symbol('x')), Integer(-1))), Integer(-1))")
print(f)
fd = diff(f)
fdd = diff(fd)

solveset(fd, x)

# for root in solveset(fd, x):
#   val = fdd.subs(x, root)
#   if val < 0:
#     print('Maximum: ', root)
#   elif val > 0:
#     print('Minimum: ', root)

# print('Wendepunkte:', solveset(fdd, x))

-1 + 2/x + 2/(x*sin(x))


Complement(ConditionSet(x, Eq(-2*x*cos(x) - 2*sin(x)**2 - 2*sin(x), 0), Complexes), Union(ImageSet(Lambda(_n, 2*_n*pi + pi), Integers), ImageSet(Lambda(_n, 2*_n*pi), Integers)))

## Integration

Generell kann das Integral einer Funktion mit `integrate` bestimmt werden. Durch den Parameter `risch=True` und der Überprüfung dessen Typs kann gleichzeitig bestimmt werden, ob die Stammfunktion noch elementar ist, siehe [Sympy Docs](https://docs.sympy.org/latest/modules/integrals/integrals.html#sympy.integrals.risch.NonElementaryIntegral).

In [4]:
a = integrate(x*exp(x**2), x, risch=True)
print(type(a))
a

<class 'sympy.core.mul.Mul'>


exp(x**2)/2

In [98]:
Poly(x**2 - 1, x).exquo(Poly(x - 1, x))

Poly(x + 1, x, domain='ZZ')

## Asymptoten

### Vertikale Asymptoten

Hierfür bestimmen wir erst die Unstetigkeitsstellen der Funktion und beobachten dann den Grenzwert an diesen Stellen. Falls dieser (-)Unendlich ist, haben wir hier eine vertikale Asymptote...?

**Problem:** Unstetigkeitsstellen sind nicht zwingend eine endliche Liste, daher gibt es nicht zwingend eine endliche Anzahl an Asymptoten

In [55]:
f = 1/x
for s in singularities(f, x):
    print(limit(f, x, s, '+'))
    print(limit(f, x, s, '-'))

oo
-oo


### Horizontale Asymptoten

Hierfür bestimmen wir den Grenzwert bei Unendlich und -Unendlich. Falls dabei reelle Zahlen rauskommen, haben wir dort horizontale Asymptoten.

In [48]:
f = 1+4*(x**2-1)/x**4
print(f)
l1 = limit(f, x, oo)
l2 = limit(f, x, -oo)

print(l1, l2)

g = 1+sin(5*x)/(2*x)
print(g)
l1 = limit(g, x, oo)
l2 = limit(g, x, -oo)

print(l1, l2)

1 + (4*x**2 - 4)/x**4
1 1
1 + sin(5*x)/(2*x)
1 1


In [63]:
f = (3*x**3 - 1) / (x**2 + 2*x)
g = x**2 + 2*x
f.as_poly(x)
g.as_poly(x)
# limit(f, x, oo)
# Poly(f).coeffs()